## StudentDataGenerator_Python_SQL

In [ ]:
import random
from faker import Faker
import mysql.connector

# Connect to the database
db = mysql.connector.connect(
    host="enterhost",
    user="enterSQLusername",
    password="enterpassword",
    database="enterdatabase"
)

# Create a cursor
cursor = db.cursor()

# Drop the students_data table if it exists
cursor.execute("DROP TABLE IF EXISTS students_data")

# Create the students_data table
cursor.execute("""
CREATE TABLE students_data (
    id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    student_id VARCHAR(255) UNIQUE,
    name VARCHAR(255),
    age INT,
    gender VARCHAR(255),
    city VARCHAR(255),
    country VARCHAR(255),
    degree VARCHAR(255),
    specialisation VARCHAR(255),
    special_classes VARCHAR(255),
    grade FLOAT
)
""")

# Function to insert student data into the database
def insert_student_data(num_rows):
    fake = Faker()
    for i in range(num_rows):
        student_id = 'S' + str(random.randint(10000, 99999))
        while True:
            # Check if the generated student ID already exists in the database
            cursor.execute("SELECT student_id FROM students_data WHERE student_id = %s", (student_id,))
            result = cursor.fetchone()
            if result is None:
                # If the student ID does not exist, break out of the loop
                break
            # If the student ID already exists, generate a new one
            student_id = 'S' + str(random.randint(10000, 99999))
        name = fake.name()
        age = random.randint(18, 25)
        gender = random.choice(['Male', 'Female'])
        city = fake.city()
        country = fake.country().replace("'", "''")
        degree = random.choice(['Bachelor of Science', 'Bachelor of Arts', 'Bachelor of Commerce'])
        specialisation = random.choice(['Computer Science', 'Marketing', 'Accounting','HR','Digital Marketing'])
        special_classes = random.choice(['Yes', 'No'])
        grade = round(random.uniform(0.0, 4.0), 2)
        query = "INSERT INTO students_data (student_id, name, age, gender, city, country, degree, specialisation, special_classes, grade) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"
        values = (student_id, name, age, gender, city, country, degree, specialisation, special_classes, grade)
        cursor.execute(query, values)

# Get user input for number of rows to insert
num_rows = int(input("Enter the number of rows of student data to insert: "))

# Insert student data in batches of 1000 rows at a time
num_iterations = num_rows // 100000
if num_rows % 100000 != 0:
    num_iterations += 1

index = 1
for i in range(num_iterations):
    insert_student_data(min(100000, num_rows))
    num_rows -= 50000
    
    # Add index to inserted rows
    cursor.execute("SELECT COUNT(*) FROM students_data")
    count = cursor.fetchone()[0]
    for j in range(index, count+1):
        cursor.execute("UPDATE students_data SET id=%s WHERE id=%s", (j, j))
    index = count + 1

# Commit the changes
db.commit()

# Close the connection
db.close()
